In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn.model_selection  # import train_test_split
import sklearn.metrics  # import accuracy_score
import sklearn.feature_extraction.text  # import CountVectorizer
import xgboost  # import XGBClassifier
import pandas as pd
import os
import numpy as np

import sys

# sys.path.append("/p/home/jusers/ehlert1/juwels/notebooks/bootcamp_testing/scripts")
sys.path.append("../../scripts")
import normalize_text_bootcamp
import utils_bootcamp
import dataset_bootcamp
import plotting
import embeddings_bootcamp

In [ ]:
FOLDER_TO_TWEETS = "/p/project/training2223/a2/data/tweets/tweets_2017_normalized_filtered.nc"
FOLDER_TO_TWEETS = "../../../data/tweets/tweets_2017-2020_normalized_filtered.nc"

In [ ]:
ds_tweets = dataset_bootcamp.load_tweets_dataset(FOLDER_TO_TWEETS)

In [ ]:
np.min(ds_tweets.latitude), np.max(ds_tweets.latitude), np.min(ds_tweets.longitude), np.max(ds_tweets.longitude)

In [ ]:
# embeddings assume lower case -> convert dataset
ds_lower_case = normalize_text_bootcamp.normalize_filter_dataset(
    ds_tweets, use_lower_case=True, do_split_punctutation_text=True
)

In [ ]:
ds_lower_case.text_normalized

In [ ]:
# using first 10_001 values to iterate more quickly + resetting index
ds_sel = ds_lower_case.sel(index=slice(0, 10_000))
ds_sel = dataset_bootcamp.reset_index_coordinate(ds_sel)

In [ ]:
ds_sel.raining.plot.hist()

In [ ]:
import re

text = ". . . -0.1573 -0.29517 0.30453 -0.54773 0.098293 -0.1776 0.21662 0.19261 -0.21101 0.53788 -0.047755 0.40675 0.023592 -0.32814 0.046858 0.19367 0.25565 -0.021019 -0.15957 -0.1023 0.20303 -0.043333 0.11618 -0.18486 0.0011948 -0.052301 0.34587 0.052335 0.16774 -0.21384 0.055947 0.24934 -0.12179 0.16749 0.28922 -0.033739 0.3015 -0.13241 0.092635 0.37155 -0.2884 -0.0052731 -0.001005 -0.51153 -0.28476 -0.20139 0.11837 -0.0055891 0.43604 0.16796 -0.2701 0.063957 -0.093253 -0.22079 0.36501 0.06545 0.23941 -0.19292 0.098293 0.12172 -0.1168 -0.027436 0.20507 -0.39139 -0.23111 0.46239 0.22888 -0.028415 -0.1798 0.23817 0.28093 -0.47935 0.23177 -0.35587 0.14246 0.11861 0.011018 0.091986 0.0054809 -0.39955 -0.40183 -0.10629 -0.30851 0.12383 -0.16737 -0.43569 0.4211 -0.57416 -0.19964 0.51312 0.090747 -0.21657 0.043519 0.24288 0.081134 0.49104 -0.33342 -0.31056 -0.3136 0.26931 -0.14402 0.33185 -0.21662 -0.072985 0.080603 -0.7266 -0.098385 -0.36233 -0.25346 0.1154 0.25738 0.15802 -0.15633 -0.024581 0.35673 0.31153 0.33475 -0.081155 -0.3061 0.019077 -0.049047 -0.11232 -0.07417 0.35596 -0.2642 0.012781 -0.20715 0.020223 0.054534 -0.28803 0.42863 -0.10312 0.24771 0.013196 0.19768 -0.013528 -0.15134 0.20307 -0.028973 -0.022706 -0.29199 -0.082062 0.19048 0.0053574 0.14067 -0.28675 0.21343 0.42428 -0.28186 -0.11801 -0.45227 -0.0067998 0.044784 -0.0062886 0.25087 0.34481 -0.64459 -0.20467 0.35007 0.1468 -0.14007 -0.0050219 -0.24053 0.41426 -0.40902 0.21141 0.25726 -0.4883 0.027066 0.56367 -0.39594 -0.035206 0.63079 0.14343 0.038315 0.32527 -0.080335 -0.20065 -0.30848 -0.0031591 0.15296 -0.21014 0.42143 -0.20944 -0.069285 0.13555 -0.020401 -0.22555 0.33491 0.16035 0.17739 -0.023627 0.097575 -0.19395 -0.018754 -0.119 -0.0067027 -0.4178 0.29027 0.13034 -0.30212 0.61173 -0.39918 -0.020191 -0.34531 -0.092082 0.46818 0.36671 0.21021 -0.053162 -0.37872 -0.14271 -0.13604 0.31715 -0.17227 -0.091266 0.16417 0.15069 0.53556 -0.29678 0.13965 -0.29788 0.1282 0.1971 -0.045515 -0.41355 -0.050333 -0.39015 -0.29579 -0.096145 -0.03151 0.053714 -0.37309 -0.36523 -0.17235 0.39251 -0.065909 -0.25267 -0.34448 -0.11503 0.43665 0.18832 0.20631 0.27801 -0.046077 0.13397 -0.091953 -0.098542 0.15811 0.2752 0.081383 0.32077 -0.10028 0.1088 -0.24836 0.10477 0.15243 -0.071302 0.12861 0.23061 0.0074864 0.090918 -0.12269 -0.14831 0.010586 0.35745 -0.23412 -0.23746 -0.22646 -0.27641 -0.1634 0.071909 -0.093884 0.21331 -0.20627 0.44406 0.34691 0.019064 0.034657 0.36789 0.32276 -0.31099 -0.023443 -0.77048 -0.26001 0.033961 -0.13874 0.051973 -0.0090509 0.27427 0.046548 -0.48214 -0.1437 -0.1975 -0.038126 -0.16555 0.071697 0.049449 0.15386 -0.81663"
print(text.__repr__())
text = text.split(" ")
word = text[:-300]
values = text[-300:]
print(values[-1])
" ".join(word), values

In [ ]:
x = "-0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.34709 0.29964 -0.66514 0.002516 -0.42333 0.27512 0.36012 0.16311 0.23964 -0.05923 0.3261 0.20559 0.038677 -0.045816 0.089764 0.43151 -0.15954 0.08532 -0.26572 -0.15001 0.084286 -0.16714 -0.43004 0.060807 0.13121 -0.24112 0.66554 0.4453 -0.18019 -0.13919 0.56252 0.21457 -0.46443 -0.012211 0.029988 -0.051094 -0.20135 0.80788 0.47377 -0.057647 0.46216 0.16084 -0.20954 -0.05452 0.15572 -0.13712 0.12972 -0.011936 -0.003378 -0.13595 -0.080711 0.20065 0.054056 0.046816 0.059539 0.046265 0.17754 -0.31094 0.28119 -0.24355 0.085252 -0.21011 -0.19472 0.0027297 -0.46341 0.14789 -0.31517 -0.065939 0.036106 0.42903 -0.33759 0.16432 0.32568 -0.050392 -0.054297 0.24074 0.41923 0.13012 -0.17167 -0.37808 -0.23089 -0.019477 -0.29291 -0.30824 0.30297 -0.22659 0.081574 -0.18516 -0.21408 0.40616 -0.28974 0.074174 -0.17795 0.28595 -0.039626 -0.2339 -0.36054 -0.067503 -0.091065 0.23438 -0.0041331 0.003232 0.0072134 0.008697 0.21614 0.049904 0.35582 0.13748 0.073361 0.14166 0.2412 -0.013322 0.15613 0.083381 0.088146 -0.019357 0.43795 0.083961 0.45309 -0.50489 -0.10865 -0.2527 -0.18251 0.20441 0.13319 0.1294 0.050594 -0.15612 -0.39543 0.12538 0.24881 -0.1927 -0.31847 -0.12719 0.4341 0.31177 -0.0040946 -0.2094 -0.079961 0.1161 -0.050794 0.015266 -0.2803 -0.12486 0.23587 0.2339 -0.14023 0.028462 0.56923 -0.1649 -0.036429 0.010051 -0.17107 -0.042608 0.044965 -0.4393 -0.26137 0.30088 -0.060772 -0.45312 -0.19076 -0.20288 0.27694 -0.060888 0.11944 0.62206 -0.19343 0.47849 -0.30113 0.059389 0.074901 0.061068 -0.4662 0.40054 -0.19099 -0.14331 0.018267 -0.18643 0.20709 -0.35598 0.05338 -0.050821 -0.1918 -0.37846 -0.06589"
y = x.split(" ")
len(np.asarray(y, dtype="float64"))

In [ ]:
# load the whole GloVe embedding into a dictionary
FOLDER_GLOVE_EMBEDDINGS = "/p/project/deepacf/maelstrom/ehlert1/embeddings/glove.6B/glove.6B.100d.txt"
FOLDER_GLOVE_EMBEDDINGS = "../../../data/embeddings/glove.840B.300d/glove.840B.300d.txt"


def load_glove_embeddings(read_lines=False):
    GLOVE_EMB_DICT = {}
    with open(
        FOLDER_GLOVE_EMBEDDINGS,
        "r",
        encoding="utf8",
    ) as f:
        if read_lines:
            lines = f.readlines()
        else:
            lines = f
        for line in lines:
            if read_lines:
                text = line.split(" ")
                word = text[:-300]
                values = text[-300:]
                word = " ".join(word)
                try:
                    coefs = np.asarray(values, dtype="float64")
                except Exception as e:
                    print(f"{e}")
                    print(line)
                    print(values)
                    print(word)
                    print(values[0])
                    print(values[1:])
                    break
            else:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype="float64")
            GLOVE_EMB_DICT[word] = coefs
    f.close()
    print("Loaded %s word vectors." % len(GLOVE_EMB_DICT))
    return GLOVE_EMB_DICT


glove_embeddings = load_glove_embeddings(read_lines=True)

In [ ]:
len(glove_embeddings)

In [ ]:
def get_embedded_data(ds, style="count_vectorizer", glove_embeddings=None):
    if style == "count_vectorizer":
        cv = sklearn.feature_extraction.text.CountVectorizer(
            max_features=5000,
            encoding="utf-8",
            ngram_range=(1, 3),
            # token_pattern = "[A-Za-z_][A-Za-z\d_]*"
        )
    elif style == "glove_twitter":
        if glove_embeddings is None:
            glove_embeddings = load_glove_embeddings()
        cv = sklearn.feature_extraction.text.CountVectorizer(
            encoding="utf-8",
            ngram_range=(1, 3),
            vocabulary=glove_embeddings,
        )
    X = cv.fit_transform(list(ds["text_normalized"].values)).toarray()
    return X


X = get_embedded_data(ds_sel, style="glove_twitter", glove_embeddings=glove_embeddings)

In [ ]:
def check_embedding_coverage(ds, glove_embeddings):
    (
        sorted_oov,
        vocab_coverage,
        text_coverage,
        vocab,
    ) = embeddings_bootcamp.check_embeddings_coverage(ds["text_normalized"].values, glove_embeddings)
    print("GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text".format(vocab_coverage, text_coverage))
    return sorted_oov, vocab_coverage, text_coverage, vocab


sorted_oov, vocab_coverage, text_coverage, vocab = check_embedding_coverage(ds_sel, glove_embeddings)

In [ ]:
ds_sel.text_normalized

In [ ]:
sorted_oov

In [ ]:
ds_sel = a2.preprocess.normalize_text.normalize_text_dataset(
    ds_sel,
    text_key="text",
    backup_text_to_key="text_normalized_original",
    use_lower_case=True,
    ignore_non_ascii=True,
    remove_punctuations="all",
    replace_keyword_emojis=False,
)

In [ ]:
sorted_oov, vocab_coverage, text_coverage, vocab = check_embedding_coverage(ds_sel, glove_embeddings)

In [ ]:
size_entity = np.array([len(x.split(" ")) for x in ds_sel.text.values])
max(size_entity)

In [ ]:
y = ds_sel["raining"].values

In [ ]:
X_train_index, X_test_index, y_train, y_test = sklearn.model_selection.train_test_split(
    np.arange(np.shape(X)[0]), y, test_size=0.33, random_state=0
)

In [ ]:
model = xgboost.XGBClassifier(n_jobs=20, early_stopping_rounds=10, random_state=0)
model.fit(X[X_train_index], y_train, eval_set=[(X[X_test_index], y_test)])

In [ ]:
model.best_iteration

In [ ]:
y_pred_prob = model.predict_proba(X[X_test_index])

In [ ]:
y_pred = model.predict(X[X_test_index])

In [ ]:
report = a2.plotting.analysis.check_prediction(y_test, y_pred)
print(report)
# print(f"f1 not raining {report['not raining']['f1-score']}; f1 raining {report['raining']['f1-score']}")

In [ ]:
a2.plotting.analysis.plot_roc(y_test, y_pred_prob[:, 1])